In [1]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.layers import Input, Activation, Flatten, Dense, Dropout,Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
import time
from tensorflow.keras.datasets import cifar10
(img_train, label_train), (img_test, label_test) = cifar10.load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [2]:
x_train = img_train.astype('float32') / 255
x_test = img_test.astype('float32') / 255
y_train = to_categorical(label_train)
y_test = to_categorical(label_test)

In [4]:
model_new = load_model('/content/drive/MyDrive/Do_an-NCKH/model_Conv2D.h5', compile=False)

In [5]:
model_new.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 15, 15, 16)        4624      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 16)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 8, 8, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0

In [7]:
img_size = 32
argx = np.random.randint(0,x_test.shape[0])
x_ = x_test[argx].reshape(1,img_size,img_size,3)
y_predict_no_Dense = model_new.predict(x_)

#Server

In [9]:
s = time.time()
w0, b0 = model_new.layers[0].get_weights()
model_ = Sequential()
for layer in model_new.layers[1:]:
  model_.add(layer)
model_.build(input_shape = model_new.layers[0].output_shape)
listw = []
for d in np.arange(w0.shape[-1]):
  w_reshape = w0[:,:,:,d].reshape(1,-1)
  w_plus = [np.random.rand() for i in range(w_reshape.shape[-1])]
  listw.append([w_reshape[0],w_plus])
w = np.array(listw)
k = np.array([[np.random.rand(),np.random.rand()],
              [np.random.rand(),np.random.rand()]])
kd = np.linalg.inv(k)
listKW = []
for i in np.arange(w.shape[0]):
    listKW.append(k.dot(w[i]))
kw = np.array(listKW)
print('\nTime: ',time.time() - s)


Time:  0.04399895668029785


In [10]:
kw.shape

(32, 2, 27)

#Client

In [12]:
s = time.time()
strd, kernel = 1, 3
listXX = []
for _x in np.arange(x_.shape[0]):
  listX = []
  for h in np.arange(0,x_.shape[1],strd):
    list_width = []
    for width in np.arange(0,x_.shape[2],strd):
      width_reshape = x_[_x,h:h+kernel,width:width+kernel,:].reshape(1,-1)
      x_plus = [np.random.rand() for i in range(width_reshape.shape[-1])]
      list_width.append([width_reshape[0], x_plus])
      if width+kernel == x_.shape[2]:
        break
    listX.append(list_width)
    if h+kernel == x_.shape[1]:
      break
  listXX.append(listX)
ex = np.array(listXX)
listKWXX = []
for xx in np.arange(ex.shape[0]):
  listKWX = []
  for h in np.arange(ex.shape[1]):
    listwi = []
    for wi in np.arange(ex.shape[2]):
      lw = []
      for ww_ in np.arange(kw.shape[0]):
        lw.append(kw[ww_].dot(ex[xx,h,wi].T))
      listwi.append(lw)
    listKWX.append(listwi)
  listKWXX.append(listKWX)
print('\nTime: ',time.time() - s)
kwx = np.array(listKWXX)
kwx.shape


Time:  0.09017586708068848


(1, 30, 30, 32, 2, 2)

#Server

In [14]:
def Relu(x):
  return np.maximum(x,0)

In [15]:
s = time.time()
oy = np.zeros(kwx.shape[0:4])
for xx in np.arange(oy.shape[0]):
  for height in np.arange(oy.shape[1]):
    for width in np.arange(oy.shape[2]):
      for dept in np.arange(oy.shape[3]):
        oy[xx,height,width,dept] = Relu(kd.dot(kwx[xx,height,width,dept])[0][0]+b0[dept])
print('Model dự đoán: ',model_.predict(oy).argmax())

print('\nTime: ',time.time() - s)

Model dự đoán:  1

Time:  0.25708627700805664


In [16]:
s = time.time()
print('Model dự đoán: ',model_.predict(oy).argmax())
print('\nTime: ',time.time() - s)

Model dự đoán:  1

Time:  0.04770803451538086
